<a href="https://www.kaggle.com/code/aliknot/facial-emotion-recognitions?scriptVersionId=232682889" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Emotion detection in Facial Recognition

## Clone github repo

In [1]:
!git clone https://github.com/aliknot/facial-emotion-recognition

Cloning into 'facial-emotion-recognition'...
remote: Enumerating objects: 863, done.
remote: Counting objects: 100% (344/344), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 863 (delta 8), reused 340 (delta 8), pack-reused 519 (from 2)
Receiving objects: 100% (863/863), 599.02 MiB | 48.29 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (161/161), done.


## Install essential libraries

In [2]:
!pip install mtcnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


## Import necessary libraries

In [3]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from mtcnn import MTCNN
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

## Global variables

In [19]:
output_base_url = '/kaggle/working/'
repo_base_url = os.path.join(output_base_url, 'facial-emotion-recognition')
images_url = os.path.join(repo_base_url, 'images')
repvgg_url = os.path.join(repo_base_url, 'RepVGG')

organized_images_url = os.path.join(output_base_url, 'images_organized')
cropped_images_url = os.path.join(output_base_url, 'cropped_images')
resized_images_url = os.path.join(output_base_url, 'resized_images')
normalized_images_url = os.path.join(output_base_url, 'normalized_images')
augmented_images_url = os.path.join(output_base_url, 'augmented_images')
splitted_images_url = os.path.join(output_base_url, 'splitted_images')

In [5]:
import sys
sys.path.append(repvgg_url)

In [6]:
# List of emotions
emotions = ['Happy', 'Neutral', 'Sad', 'Surprised', 'Disgust', 'Anger', 'Fear', 'Contempt']

## Organize the photos in emotions folders

In [7]:
if not os.path.exists(organized_images_url):
    os.makedirs(organized_images_url)
    
# Iterate through each numbered folder (0-18)
for folder_name in sorted(os.listdir(images_url)):
    folder_path = os.path.join(images_url, folder_name)
    if os.path.isdir(folder_path):  # Ensure it's a directory
        
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            
            if os.path.isfile(img_path):  # Ensure it's a file
                emotion_label = img_file.split('.')[0]  # Extract the emotion label
                
                if emotion_label in emotions:
                    destination_folder = os.path.join(organized_images_url, emotion_label)
                    if not os.path.exists(destination_folder):
                        os.makedirs(destination_folder)
                    
                    # Rename the image with folder name to avoid duplicates
                    new_img_name = f"{folder_name}_{img_file}"
                    destination_path = os.path.join(destination_folder, new_img_name)
                    
                    shutil.copy(img_path, destination_path)  # Copy instead of move
                    print(f"Copied {img_file} to {destination_path}")

print("Dataset successfully organized!")

Copied Surprised.jpg to /kaggle/working/images_organized/Surprised/0_Surprised.jpg
Copied Anger.jpg to /kaggle/working/images_organized/Anger/0_Anger.jpg
Copied Sad.jpg to /kaggle/working/images_organized/Sad/0_Sad.jpg
Copied Neutral.jpg to /kaggle/working/images_organized/Neutral/0_Neutral.jpg
Copied Contempt.jpg to /kaggle/working/images_organized/Contempt/0_Contempt.jpg
Copied Happy.jpg to /kaggle/working/images_organized/Happy/0_Happy.jpg
Copied Fear.jpg to /kaggle/working/images_organized/Fear/0_Fear.jpg
Copied Disgust.jpg to /kaggle/working/images_organized/Disgust/0_Disgust.jpg
Copied Surprised.jpg to /kaggle/working/images_organized/Surprised/1_Surprised.jpg
Copied Anger.jpg to /kaggle/working/images_organized/Anger/1_Anger.jpg
Copied Sad.jpg to /kaggle/working/images_organized/Sad/1_Sad.jpg
Copied Neutral.jpg to /kaggle/working/images_organized/Neutral/1_Neutral.jpg
Copied Contempt.jpg to /kaggle/working/images_organized/Contempt/1_Contempt.jpg
Copied Happy.jpg to /kaggle/work

## Put the organized dataset in df

In [8]:
# Initialize an empty list to store data
data = []

# Iterate over emotion folders
for emotion in emotions:
    folder_path = os.path.join(organized_images_url, emotion)
    if os.path.exists(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.lower().endswith('.jpg'):
                image_path = os.path.join(folder_path, file_name)
                folder_name = file_name.split('_')[0]  # Extracting folder number from filename
                
                data.append({
                    'image_path': image_path,
                    'emotion': emotion,
                    'folder': folder_name
                })

# Create DataFrame
df = pd.DataFrame(data)

# Display first few rows
df.head()

,image_path,emotion,folder
0,/kaggle/working/images_organized/Happy/10_Happ...,Happy,10
1,/kaggle/working/images_organized/Happy/9_Happy...,Happy,9
2,/kaggle/working/images_organized/Happy/12_Happ...,Happy,12
3,/kaggle/working/images_organized/Happy/5_Happy...,Happy,5
4,/kaggle/working/images_organized/Happy/4_Happy...,Happy,4


## Visualize all of the imgaes per emotion

In [9]:
"""
# Count number of images per emotion
emotion_counts = df['emotion'].value_counts()

# Plot the data
plt.figure(figsize=(10, 5))
sns.barplot(x=emotion_counts.index, y=emotion_counts.values, palette="viridis", hue=emotion_counts.index, legend=False)

# Formatting
plt.title("Number of Images per Emotion", fontsize=14)
plt.xlabel("Emotion", fontsize=12)
plt.ylabel("Number of Images", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()
"""

'\n# Count number of images per emotion\nemotion_counts = df[\'emotion\'].value_counts()\n\n# Plot the data\nplt.figure(figsize=(10, 5))\nsns.barplot(x=emotion_counts.index, y=emotion_counts.values, palette="viridis", hue=emotion_counts.index, legend=False)\n\n# Formatting\nplt.title("Number of Images per Emotion", fontsize=14)\nplt.xlabel("Emotion", fontsize=12)\nplt.ylabel("Number of Images", fontsize=12)\nplt.xticks(rotation=45)\nplt.grid(axis=\'y\', linestyle=\'--\', alpha=0.7)\n\n# Show the plot\nplt.show()\n'

## Crop all of the images, focusing on the face

In [10]:
# Initialize MTCNN detector
detector = MTCNN(device = 'GPU')

# Check if the the images are already cropped or not
if not os.path.exists(cropped_images_url):
    os.makedirs(cropped_images_url)
    
    # Process each emotion folder
    for emotion in emotions:
        emotion_folder = os.path.join(organized_images_url, emotion)
        cropped_emotion_folder = os.path.join(cropped_images_url, emotion)
        
        os.makedirs(cropped_emotion_folder, exist_ok=True)
    
        for img_name in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_name)
            image = cv2.imread(img_path)
    
            if image is None:
                print(f"Skipping {img_name}, not a valid image.")
                continue
    
            # Convert image to RGB for MTCNN
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
            # Detect faces
            detections = detector.detect_faces(image_rgb)
    
            if len(detections) == 0:
                print(f"No face detected in {img_name}, skipping...")
                continue
    
            # Get the bounding box of the first detected face
            x, y, w, h = detections[0]['box']
            cropped_face = image[y:y+h, x:x+w]
    
            # Save cropped image
            cropped_img_path = os.path.join(cropped_emotion_folder, img_name)
            cv2.imwrite(cropped_img_path, cropped_face)
            print("Cropped the image", img_name, emotion)

    print("Face cropping completed!")
else:
    print("Images are already cropped!")

Cropped the image 10_Happy.jpg Happy
Cropped the image 9_Happy.jpg Happy
Cropped the image 12_Happy.jpg Happy
Cropped the image 5_Happy.jpg Happy
Cropped the image 4_Happy.jpg Happy
Cropped the image 0_Happy.jpg Happy
Cropped the image 2_Happy.jpg Happy
Cropped the image 3_Happy.jpg Happy
Cropped the image 1_Happy.jpg Happy
Cropped the image 7_Happy.jpg Happy
Cropped the image 15_Happy.jpg Happy
Cropped the image 18_Happy.jpg Happy
Cropped the image 8_Happy.jpg Happy
Cropped the image 6_Happy.jpg Happy
Cropped the image 13_Happy.jpg Happy
Cropped the image 17_Happy.jpg Happy
Cropped the image 14_Happy.jpg Happy
Cropped the image 11_Happy.jpg Happy
Cropped the image 16_Happy.jpg Happy
Cropped the image 2_Neutral.jpg Neutral
Cropped the image 9_Neutral.jpg Neutral
Cropped the image 1_Neutral.jpg Neutral
Cropped the image 13_Neutral.jpg Neutral
Cropped the image 18_Neutral.jpg Neutral
Cropped the image 16_Neutral.jpg Neutral
Cropped the image 7_Neutral.jpg Neutral
Cropped the image 6_Neut

## Resize Images

In [11]:
# Target size for RepVGG
target_size = (224, 224)

# Check if the the images are already resized or not
if not os.path.exists(resized_images_url):
    os.makedirs(resized_images_url)
    
    # Process each emotion folder
    for emotion in emotions:
        emotion_folder = os.path.join(cropped_images_url, emotion)
        resized_emotion_folder = os.path.join(resized_images_url, emotion)
        
        os.makedirs(resized_emotion_folder, exist_ok=True)
    
        for img_name in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_name)
            image = cv2.imread(img_path)
    
            if image is None:
                print(f"Skipping {img_name}, not a valid image.")
                continue
    
            # Resize the image
            resized_image = cv2.resize(image, target_size)
    
            # Save the resized image
            resized_img_path = os.path.join(resized_emotion_folder, img_name)
            cv2.imwrite(resized_img_path, resized_image)

    print("Image resizing completed! All images are now 224x224.")
else:
    print("Images are already resized!")

Image resizing completed! All images are now 224x224.


## Normalize Images

In [12]:
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        for root, _, files in os.walk(image_dir):
            for file in files:
                if file.endswith(('png', 'jpg', 'jpeg')):
                    self.image_paths.append(os.path.join(root, file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Define a transform to convert images to tensors without normalization
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create the dataset and dataloader
dataset = ImageDataset(image_dir=resized_images_url, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

# Function to calculate mean and std
def calculate_mean_std(loader):
    mean = 0.0
    std = 0.0
    total_samples = 0
    for images in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_samples += batch_samples

    mean /= total_samples
    std /= total_samples
    return mean, std

# Calculate mean and std
mean, std = calculate_mean_std(dataloader)
print(f"Calculated Mean: {mean}")
print(f"Calculated Std: {std}")

Calculated Mean: tensor([0.6104, 0.4769, 0.4438])
Calculated Std: tensor([0.1794, 0.1767, 0.1775])


In [13]:
# Define the normalization transform
normalize = transforms.Normalize(mean=mean, std=std)

# Define the complete transform pipeline
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

# Remove the directory if it exists to avoid duplicates
if os.path.exists(normalized_images_url):
    shutil.rmtree(normalized_images_url)
os.makedirs(normalized_images_url)

# Normalize and save images
for img_rel_path in dataset.image_paths:
    img = Image.open(img_rel_path).convert('RGB')
    img_tensor = transform(img)
    # Convert tensor back to PIL Image
    img_normalized = transforms.ToPILImage()(img_tensor)
    # Define the path to save the normalized image
    relative_path = os.path.relpath(img_rel_path, resized_images_url)
    save_path = os.path.join(normalized_images_url, relative_path)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    img_normalized.save(save_path)

print("All images have been normalized and saved.")

All images have been normalized and saved.


## Data Augmentation

In [17]:
import os
from PIL import Image
from torchvision import transforms
import torch

# Define the data augmentation pipeline
data_augmentation = transforms.Compose([
    transforms.RandomRotation(degrees=30),            # Random rotation up to 30 degrees
    transforms.RandomHorizontalFlip(p=0.5),           # Random horizontal flip with 50% probability
    transforms.RandomVerticalFlip(p=0.5),             # Random vertical flip with 50% probability
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random brightness, contrast, saturation, and hue adjustments
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation (shifts) up to 10% of image size
    transforms.ToTensor(),                            # Convert image to tensor
    transforms.Lambda(lambda x: x + 0.1 * torch.randn_like(x))  # Add random noise
])

# Number of augmented images to generate per original image
num_augmented_images = 500

# Iterate over each emotion category
for emotion in os.listdir(normalized_images_url):
    emotion_dir = os.path.join(normalized_images_url, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    # Create corresponding directory in the output folder
    output_emotion_dir = os.path.join(augmented_images_url, emotion)
    os.makedirs(output_emotion_dir, exist_ok=True)

    # Process each image in the current emotion directory
    for img_name in os.listdir(emotion_dir):
        img_path = os.path.join(emotion_dir, img_name)
        if not os.path.isfile(img_path):
            continue

        # Load the image
        image = Image.open(img_path).convert('RGB')

        # Generate augmented images
        for i in range(num_augmented_images):
            augmented_image = data_augmentation(image)
            augmented_image_pil = transforms.ToPILImage()(augmented_image)

            # Save the augmented image
            base_name, ext = os.path.splitext(img_name)
            augmented_img_name = f"{base_name}_aug_{i}{ext}"
            augmented_img_path = os.path.join(output_emotion_dir, augmented_img_name)
            augmented_image_pil.save(augmented_img_path)

print("Data augmentation complete. Augmented images are saved in:", augmented_images_url)

Data augmentation complete. Augmented images are saved in: /kaggle/working/augmented_images


## Split the Dataset

In [20]:
import os
import shutil
import random

# Set the seed for reproducibility
random.seed(1337)

# Define paths
train_dir = os.path.join(splitted_images_url, 'train')
val_dir = os.path.join(splitted_images_url, 'val')
test_dir = os.path.join(splitted_images_url, 'test')

# Define split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Create directories for splits
for dir in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)

# Iterate over each class folder
for class_name in os.listdir(augmented_images_url):
    class_dir = os.path.join(augmented_images_url, class_name)
    if os.path.isdir(class_dir):
        images = os.listdir(class_dir)
        random.shuffle(images)
        total_images = len(images)
        train_end = int(train_ratio * total_images)
        val_end = train_end + int(val_ratio * total_images)

        # Define paths for each split
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)

        # Create class directories if they don't exist
        for dir in [train_class_dir, val_class_dir, test_class_dir]:
            if not os.path.exists(dir):
                os.makedirs(dir)

        # Copy images to respective directories
        for i, image in enumerate(images):
            src = os.path.join(class_dir, image)
            if i < train_end:
                dst = os.path.join(train_class_dir, image)
            elif i < val_end:
                dst = os.path.join(val_class_dir, image)
            else:
                dst = os.path.join(test_class_dir, image)
            shutil.copyfile(src, dst)

print("Dataset successfully split into train, validation, and test sets.")

Dataset successfully split into train, validation, and test sets.


## Model

In [21]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define paths
train_dir = os.path.join(splitted_images_url, 'train')
val_dir = os.path.join(splitted_images_url, 'val')
test_dir = os.path.join(splitted_images_url, 'test')

# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),  # Convert images to tensors
        # Exclude normalization since images are pre-normalized
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        # Exclude normalization
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        # Exclude normalization
    ]),
}

# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [22]:
import torch
import torch.nn as nn
from repvgg import create_RepVGG_A0

# Initialize the RepVGG-A0 model
model = create_RepVGG_A0(deploy=False)

# Load pre-trained weights
checkpoint = torch.load('RepVGG-A0-train.pth')
model.load_state_dict(checkpoint)

# Modify the classifier to match the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.linear = nn.Linear(model.linear.in_features, num_classes)

ModuleNotFoundError: No module named 'repvgg'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")

In [ ]:
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Loss: {test_loss/len(test_loader)}, Accuracy: {100 * correct / total}%")